In [ ]:
valid_classes = [0,85, 170]
class_names = ["Background","Pupil","Iris"]


class_map = dict(zip(valid_classes, range(len(valid_classes))))
n_classes=len(valid_classes)


colors = [ [  0,   0,   0],[0,255,0],[0,0,255]]
label_colours = dict(zip(range(n_classes), colors))


In [ ]:
import cv2
import os
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
from skimage.measure import label,regionprops,find_contours

In [ ]:
img_path = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/Miche_Dataset/train_img/009_GT2_IN_F_LI_01_2.jpg"
mask_path = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/Miche_Dataset/train_masks/009_GT2_IN_F_LI_01_2.bmp"


In [ ]:
img = cv2.imread(img_path,cv2.IMREAD_COLOR) # Read Image as BGR

print(img.shape)
# h,w = img.shape[:2]
h,w = 400,400

img = cv2.resize(img,(w,h))


plt.imshow(img[:,:,::-1])



In [ ]:
mask = cv2.imread(mask_path)

mask = cv2.cvtColor(cv2.resize(mask,(w,h)),cv2.COLOR_BGR2GRAY)

print(mask.shape)
plt.imshow(mask)

In [ ]:
median = cv2.medianBlur(mask, 5)
        
edge_detected_image = cv2.Canny(median, 12, 234)


contours, hierarchy = cv2.findContours(edge_detected_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
plt.imshow(edge_detected_image)

In [ ]:
print(len(contours)) 


In [ ]:
bg_mask = np.zeros_like(img)


print(bg_mask.shape)

In [ ]:
max_area = {}
for contour in contours:
    
    approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
    area = cv2.contourArea(contour)
    # print(area)
    if ((len(approx) > 8) & (len(approx) < 23) & (area > 30) ):
        
        
        max_area[area] = contour
        

x = sorted(max_area,key = lambda x:x)
print(x)
max_contour = max_area[x[-1]]
min_contour = max_area[x[1]]

In [ ]:
cv2.drawContours(bg_mask, [max_contour],  0, (0,255,0), -1)

cv2.drawContours(bg_mask,[min_contour],0,(0,0,255),-1)


In [ ]:
plt.imshow(bg_mask)

In [ ]:
color_map = [[0,0,0],[0,255,0],[0,0,255]]

In [ ]:
# area = cv2.contourArea(min_contour)
# print(area)

print(np.unique(bg_mask))

In [ ]:
output_mask = []
for i,color in enumerate(color_map):
    cmap = np.all(np.equal(bg_mask,color),axis=-1)
    output_mask.append(cmap)
output_mask = np.stack(output_mask,axis=-1)

In [ ]:
grayscale_mask = np.argmax(output_mask, axis=-1)
grayscale_mask = (grayscale_mask / len(color_map)) * 255

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(grayscale_mask)
cv2.imwrite("Inspection_1.png",grayscale_mask)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])
plt.imshow(grayscale_mask,alpha=0.5)

In [ ]:
resized_image = cv2.resize(img,(64,64))

resized_mask = cv2.resize(grayscale_mask,(256,256))


resized_image = cv2.resize(resized_image,(256,256))

In [ ]:
# Mask Changes
# if masks are resized into image sizes then shape of the mask won't change even mask and image have different image dimensions.
# if masks and images are resized into fixed image sizes ,mask shape won't change


In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(resized_mask)

In [ ]:
cv2.imwrite("Inspection_2.png",resized_mask)

In [ ]:
approx_max = cv2.approxPolyDP(max_contour,0.001*cv2.arcLength(max_contour,True),True)
approx_min =  cv2.approxPolyDP(min_contour,0.001*cv2.arcLength(min_contour,True),True)

In [ ]:
print(cv2.contourArea(max_contour))
print(cv2.contourArea(approx_max))
print(cv2.contourArea(min_contour))
print(cv2.contourArea(approx_min))

In [ ]:
bg_up_1 = np.zeros_like(img)
# print(bg_up_1.shape)

In [ ]:
cv2.drawContours(bg_up_1 , [approx_max],  0, (0,255,0), cv2.FILLED)

cv2.drawContours(bg_up_1 ,[approx_min],0,(0,0,255),cv2.FILLED) 

In [ ]:
plt.imshow(bg_up_1)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])
plt.imshow(bg_up_1,alpha=0.5)

In [ ]:
cv2.imwrite("Inspection_3.png",bg_up_1)

In [ ]:
cv2.imwrite("Inspection_4.png",cv2.resize(bg_up_1,(256,256)))

In [ ]:
from shapely.geometry import Polygon


In [ ]:
poly_1 = []
for coords in approx_max:
    # print(coords[0])
    poly_1.append(coords[0])

In [ ]:
poly_2 = []
for coords in approx_min:
    # print(coords[0])
    poly_2.append(coords[0])

In [ ]:
polygon1 = Polygon(poly_1)
polygon2 = Polygon(poly_2)

In [ ]:
difference = polygon1.difference(polygon2)

In [ ]:
approx_max.shape

In [ ]:
approx_max[0]

In [ ]:
intersect_part = list(difference.exterior.coords)

In [ ]:
cont_up = []
for coords in intersect_part:
    cont_up.append([coords])
    

In [ ]:
cont_up = np.array(cont_up).astype(np.int32)

In [ ]:
cont_up.dtype

In [ ]:
intersect_part[0]

In [ ]:

bg_up_2 = np.zeros_like(img)

In [ ]:
# cv2.drawContours(bg_up_2 , [cont_up],  0, (0,255,0), -1)

In [ ]:
plt.imshow(bg_up_2)

In [ ]:
from PIL import Image,ImageFilter

mask_path = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/Miche_Dataset/train_masks/003_GS4_IN_R_RI_01_1.bmp"

img = Image.open(mask_path)

In [ ]:
# im_5 = im_1.filter(ImageFilter.EDGE_ENHANCE) 
im_6 = img.filter(ImageFilter.EDGE_ENHANCE_MORE) 

In [ ]:
im_6.save("EdgeEnhanced_Image.png")

In [ ]:
mask = cv2.imread("/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Experiments/utils/Masks_with_256/075_GT2_OU_F_LI_01_1.png",0)
img = cv2.imread("/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/Miche_Dataset/train_img/075_GT2_OU_F_LI_01_1.jpg")

In [ ]:
img = cv2.resize(img,(256,256))

In [ ]:
# plt.imshow(img[:,:,::-1])

mask = decode_segmap(mask)

In [ ]:

plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])
plt.imshow(mask,alpha=0.5)

In [ ]:
plt.imshow(img[:,:,::-1])

In [ ]:

colors = [ [  0,   0,   0],[0,255,0],[0,0,255]]
label_colours = dict(zip(range(n_classes), colors))

valid_classes = [0,85, 170]
class_names = ["Background","Pupil","Iris"]


class_map = dict(zip(valid_classes, range(len(valid_classes))))
n_classes=len(valid_classes)


def decode_segmap(temp):
    #convert gray scale to color
    # temp=temp.numpy()
    r = temp.copy()
    g = temp.copy()
    b = temp.copy()
    for l in range(0, n_classes):
        r[temp == l] = label_colours[l][0]
        g[temp == l] = label_colours[l][1]
        b[temp == l] = label_colours[l][2]

    rgb = np.zeros((temp.shape[0], temp.shape[1], 3))
    rgb[:, :, 0] = r / 255.0
    rgb[:, :, 1] = g / 255.0
    rgb[:, :, 2] = b / 255.0
    return rgb


In [ ]:
mask = cv2.imread("/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/Miche_Dataset/train_masks/075_GT2_OU_F_LI_01_1.bmp")
img = cv2.imread("/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/Miche_Dataset/train_img/075_GT2_OU_F_LI_01_1.jpg")


print(img.shape)
print(mask.shape)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])
plt.imshow(mask,alpha=0.5)

In [ ]:
median = cv2.medianBlur(mask, 5)
        
edge_detected_image = cv2.Canny(median, 12, 234)


contours, hierarchy = cv2.findContours(edge_detected_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
max_area = {}
for contour in contours:
    
    approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
    area = cv2.contourArea(contour)
    # print(area)
    if ((len(approx) > 8) & (len(approx) < 23) & (area > 30) ):
        
        
        max_area[area] = contour
        

x = sorted(max_area,key = lambda x:x)
print(x)
max_contour = max_area[x[-1]]
min_contour = max_area[x[1]]

In [ ]:
approx_max = cv2.approxPolyDP(max_contour,0.001*cv2.arcLength(max_contour,True),True)
approx_min =  cv2.approxPolyDP(min_contour,0.001*cv2.arcLength(min_contour,True),True)

In [ ]:
m1 = np.zeros_like(img)

print(m1.shape)

In [ ]:
cv2.drawContours(m1 , [approx_max],  0, (0,255,0), cv2.FILLED)

cv2.drawContours(m1 ,[approx_min],0,(0,0,255),cv2.FILLED) 

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])
plt.imshow(m1,alpha=0.5)

In [ ]:
cv2.imwrite("Inspection_4.png",m1)

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import mediapipe
import pandas as pd

In [ ]:
image_dir = '/home/nipun/Pictures/My_New_Dataset'

In [ ]:

saved_dir = "./eyes_mine"


if not os.path.exists(saved_dir):
    os.makedirs(saved_dir)

In [ ]:
def mpArrayToNumpy(landmark_array, img):

    shape_arr = []

    for landmark in landmark_array.landmark:
        x = landmark.x
        y = landmark.y

        relative_x = int(x * img.shape[1])
        relative_y = int(y * img.shape[0])

        shape_arr.append([relative_x, relative_y])

    return np.array(shape_arr)


def cropped_image(img, shape_array, padded_amt=30):
    """Cropped eye region 

    Args:
        img (__numpy__): _Original Image_
        shape_array (_numpy_): _FaceLandMark locations_
        padded_amt (int, optional): _padding size_. Defaults to 15.

    """

    Leye = {"top_left": shape_array[70], "bottom_right": shape_array[133]}

    Reye = {"top_left": shape_array[285],
            "bottom_right": shape_array[446]}

    left_eye = img[Leye["top_left"][1]:Leye["bottom_right"][1] +
                   15
                   , Leye["top_left"][0]:Leye["bottom_right"][0]]

    right_eye = img[Reye["top_left"][1]:Reye["bottom_right"][1] +
                    30, Reye["top_left"][0]-5:Reye["bottom_right"][0]]
    
    
    
    Reye['top_left'][0] = Reye['top_left'][0] - 5

    return left_eye, right_eye, Leye, Reye

In [ ]:
mp_face_mesh = mediapipe.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

In [ ]:
image_count = 0
visualize = True
data_array =[]


for img_path in os.scandir(image_dir):

    
    img = cv2.imread(img_path.path)

    
    
    results = face_mesh.process(img)
    
    if results.multi_face_landmarks is not None:
        image_count +=1
        
        img_name = f'I2head_{image_count}'
        
        landmarks = results.multi_face_landmarks[0]

        shape_arr = mpArrayToNumpy(landmarks, img)

        left_eye, right_eye, Leye, Reye = cropped_image(
            img, shape_arr)
        


        cv2.imwrite(os.path.join(
            saved_dir, f"{img_name}_left.png"), left_eye)

        cv2.imwrite(os.path.join(
            saved_dir, f"{img_name}_right.png"), right_eye)

        if visualize:

            

            fig, axs = plt.subplots(1, 2)

            axs[0].set_title("Left Eye")
            axs[1].set_title("Right Eye")

            axs[0].axis("off")
            axs[1].axis("off")

            axs[0].imshow(left_eye[:, :, ::-1])
            axs[1].imshow(right_eye[:, :, ::-1])

            plt.tight_layout()
            plt.show()
            plt.close('all')

    else:
        print(
            f"MediaPipe was failed to detect the faces on the image name {img_name}")
        continue

In [ ]:
# Yolo Cropping Eye

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
from torch.utils.data import Dataset
import torch
from PIL import Image
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from glob import glob
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import cv2
import time 
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_images = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/train_img"
train_masks  = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/train_masks"


val_images = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/val_img"

val_masks =  "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/val_masks" 




In [ ]:

train_x = sorted(
        glob(f"{train_images}/*"))
train_y = sorted(
        
        glob(f"{train_masks}/*"))
valid_x = sorted(
        glob(f"{val_images}/*"))
valid_y = sorted(
        glob(f"{val_masks }/*"))


yolo_model_path = "/home/nipun/Music/yolov5/runs/train/exp8/weights/best.pt"
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_model_path ,force_reload=True)

# Filter BBOX Based on Confidence
model.conf = 0.40


saved_img_location = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/YoloCroppedEyes/images"
saved_mask_location = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/YoloCroppedEyes/masks"



if not os.path.exists(saved_img_location ):
    os.makedirs(saved_img_location)
    
    


if not os.path.exists(saved_mask_location):
    os.makedirs(saved_mask_location)

In [ ]:
def find_contours(image):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    median = cv2.medianBlur(gray, 5)
        
    edge_detected_image = cv2.Canny(median, 0, 200)
    


    contours, hierarchy = cv2.findContours(edge_detected_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [ ]:
%matplotlib inline
padded_amt = 15

pad_top_left_y1 = 20
pad_bottom_right_x2 = 7
RESIZE_AMT = 256


rows = len(valid_x)



for z, (img_path, mask_path) in enumerate(zip(train_x, train_y)):

    
    image = cv2.imread(img_path)

    h, w = image.shape[:2]
    
    mask =  cv2.imread(mask_path)
    
    
    contour_img = np.zeros_like(image)
    
    
    contours = find_contours(mask)
    
    max_area = {}
    for contour in contours:
        
        approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
        area = cv2.contourArea(contour)
        # print(area)
        if ((len(approx) > 8) & (len(approx) < 23) & (area > 30) ):
            
            
            max_area[area] = contour
            
    
    x = sorted(max_area,key = lambda x:x)

    max_contour = max_area[x[-1]]
    min_contour = max_area[x[1]]

    cv2.drawContours(contour_img ,[max_contour],0,(0,0,255),-1)

    cv2.drawContours(contour_img ,[min_contour],0,(0,255,0),-1)
    


    results = model(image[:, :, ::-1], size=640)

    df = results.pandas().xyxy[0]

    # Get the all BBOX related to Iris Class which is zero
    df = df[df["class"] == 0]

    df = df[df['confidence'] == df['confidence'].max()]

    

    for (i, row) in df.iterrows():

        x1 = round(row["xmin"])
        y1 = round(row["ymin"])
        x2 = round(row["xmax"])
        y2 = round(row["ymax"])

   
    copy_img = image.copy()

    if y1 < pad_top_left_y1:
        pad_top_left_y1 = y1

    elif x1 < padded_amt:
        padded_amt = x1
    cropped_img = copy_img[y1-pad_top_left_y1:y2 +
                                padded_amt, x1-padded_amt:x2+pad_bottom_right_x2]

    cropped_mask = contour_img[y1-pad_top_left_y1:y2 +
                                padded_amt, x1-padded_amt:x2+pad_bottom_right_x2]
    cropped_mask = np.where(cropped_mask > 0, 255, 0)

    image_name = img_path.split("/")[-1]
    mask_name = mask_path.split("/")[-1]
    
    
    
    
    draw_img = cropped_img.copy()
    
    gt_contours  = find_contours(cropped_mask.astype(np.uint8))
    
   
    
    for gt_cnt in gt_contours:
            cv2.drawContours(draw_img, [gt_cnt],  -1, (0,255,0), 1)
    
    
    
    
    
    
    plt.imshow(draw_img[:,:,::-1])
    plt.show()
   
    
    
    



In [ ]:
img_path = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/train_img/011_GT2_IN_F_LI_01_1.jpg"
img = cv2.imread(img_path)

mask_path = "/home/nipun/Documents/Uni_Malta/Datasets/Datasets/Miche/train_masks/011_GT2_IN_F_LI_01_1.bmp"
mask = cv2.imread(mask_path)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(img[:,:,::-1])

In [ ]:
plt.imshow(mask[:,:,::-1])

In [ ]:
contours = find_contours(mask)

In [ ]:
for gt_cnt in contours:
            cv2.drawContours(img, [gt_cnt],  -1, (0,255,0), 1)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(img[:,:,::-1])

In [ ]:
contour_img = np.zeros((img.shape[0],img.shape[1]))


print(contour_img.shape)

In [ ]:
%matplotlib inline


contours = np.array( [ [50,50], [50,150], [150, 150], [150,50] ] )

contours1 = np.array( [ [50,50], [50,150], [180, 150], [190,50] ] )
img = np.zeros( (200,200) ) # create a single channel 200x200 pixel black image 
cv2.fillPoly(img, pts =[contours], color=1)
cv2.fillPoly(img, pts =[contours1], color=2)


plt.imshow(img)

plt.show()







In [ ]:
contours = find_contours(mask)
big_contour = max(contours, key=cv2.contourArea)
peri = cv2.arcLength(big_contour, True)
big_contour = cv2.approxPolyDP(big_contour, 0.001 * peri, True)
cv2.drawContours(contour_img ,[big_contour], -1, (255,255,255), thickness=cv2.FILLED)

plt.imshow(contour_img )

In [ ]:
min_contour = min(contours, key=cv2.contourArea)
peri_min = cv2.arcLength(min_contour, True)
min_contour = cv2.approxPolyDP(min_contour , 0.001 * peri_min, True)

cv2.drawContours(contour_img ,[min_contour], -1, (255,0,0), thickness=cv2.FILLED)
plt.imshow(contour_img )